## Required Imports

In [14]:
import h5py
import matplotlib.pyplot as plt
import json
import numpy as np
import plotly.graph_objects as go
import lmfit as lm

from common import save_to_igor_itx, calculate_fit, i_sense, entropy_weak

from squarewave import process_per_row_parts, SquareProcessedData, process_avg_parts, get_data_part, single_wave_masks, convert_to_4_setpoint_AW
from common import bin_data, mean_data, calculate_fit, i_sense
from dataclasses import dataclass
import lmfit as lm

## Data -- Hot Cold transition

In [15]:
@dataclass
class Entry:
    x: np.ndarray
    data: np.ndarray
    aw: np.ndarray
    num_pts: int
    num_steps: int
    measure_freq: float

    processed: SquareProcessedData = None
    cold_data: np.ndarray = None
    entropy_fit: lm.model.ModelResult = None


In [16]:
datnums = [7936, 1167, 1175, 1183]

entries = []
for datnum in datnums:
    with h5py.File(f'data/dat{datnum}.h5', 'r') as f:
        data = f['cscurrent_2d'][:]
        x = f['x_array'][:]
        sweeplogs = f['metadata'].attrs['sweep_logs']
        aw = f['fdAW_0'][:]
    sweeplogs = json.loads(sweeplogs)
    awg_info = sweeplogs['FastDAC 1']['AWG']
    num_pts = awg_info['waveLen']
    num_steps = awg_info['numSteps']
    # measure_freq = sweeplogs['measureFreq']
    measure_freq = sweeplogs['FastDAC 1']['MeasureFreq']
    aw = convert_to_4_setpoint_AW(aw)
    entries.append(Entry(
        x=x,
        data=data,
        aw=aw,
        num_pts=num_pts,
        num_steps=num_steps,
        measure_freq=measure_freq
    ))

In [17]:
setpoint_averaging_delay_ms = 3  # How many ms to throw out before averaging setpoint values
for entry in entries:
    setpoint_start_index = round(setpoint_averaging_delay_ms/1000*entry.measure_freq)
    print(f'Setpoint start index = {setpoint_start_index}')
    entry.processed = process_per_row_parts(x=entry.x, data=entry.data, arbitrary_wave=entry.aw, setpoint_start=setpoint_start_index, num_steps=entry.num_steps,num_cycles=1)  # setpoint 13 = 5ms
    entry.cold_data = get_data_part(entry.processed.cycled, 'cold')



Setpoint start index = 8
Setpoint start index = 37
Setpoint start index = 37
Setpoint start index = 37


In [18]:
params = lm.Parameters()
params.add_many(
    ('mid', 0, True, None, None, None, None),
    ('theta', 5, True, 0.01, None, None, None),
    ('amp', 1, True, 0, None, None, None),
    ('lin', 0.001, True, 0, None, None, None),
    ('const', 5, True, None, None, None, None)
)

for entry in entries:
    fits = [calculate_fit(entry.x, d, params, func=i_sense) for d in entry.data]
    centers = [fit.params['mid'].value for fit in fits]
    entry.processed = process_avg_parts(entry.processed, centers)

## Zoom in on parts of transition data

In [19]:
transition_processed = entries[0].processed

x = transition_processed.x
cold = get_data_part(transition_processed.averaged, 'cold')  # Average unheated
hot = get_data_part(transition_processed.averaged, 'hot')  # Average heated
v0_0 = get_data_part(transition_processed.averaged, 0)  # First unheated
vp = get_data_part(transition_processed.averaged, 1)  # Heating with positive bias
v0_1 = get_data_part(transition_processed.averaged, 2)  # Second unheated
vm = get_data_part(transition_processed.averaged, 3)  # Heating with negative bias

# entropy = transition_processed.average_entropy_signal


In [20]:
cold_fit = calculate_fit(x, cold, params=params, func=i_sense)
hot_fit = calculate_fit(x, hot, params=params, func=i_sense)

## Entropy fits

In [21]:
for entry in entries[1:]:
    entropy_params = lm.Parameters()
    entropy_params.add_many(
        ('mid', 0, True, None, None, None, None),
        ('theta', 25, True, 0, 500, None, None),
        ('const', 0, False, None, None, None, None),
        ('dS', 0, True, -5, 5, None, None),
        ('dT', 5, True, -10, 50, None, None)
    )
    entry.entropy_fit = calculate_fit(entry.processed.x, entry.processed.average_entropy_signal, params=entropy_params, func=entropy_weak)


## Quick plot before exporting data to Igor for final Figure

In [23]:
fig = go.Figure()
fig.update_layout(title='Hot/Cold transition')
fig.add_trace(go.Scatter(x=x, y=v0_0, mode='markers', marker=dict(color='blue', symbol='circle-open')))
fig.add_trace(go.Scatter(x=x, y=v0_1, mode='markers', marker=dict(color='blue', symbol='cross')))
fig.add_trace(go.Scatter(x=x, y=cold_fit.eval(x=x), mode='lines', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=x, y=vp, mode='markers', marker=dict(color='red', symbol='circle-open')))
fig.add_trace(go.Scatter(x=x, y=vm, mode='markers', marker=dict(color='red', symbol='cross')))
fig.add_trace(go.Scatter(x=x, y=hot_fit.eval(x=x), mode='lines', line=dict(color='red')))
fig

In [24]:
entries[1].entropy_fit

In [25]:
y_spacing = 0.05

fig = go.Figure()
fig.update_layout(title='Entropy signal for several datasets')
for i, entry in enumerate(entries[1:]):
    x = entry.processed.x
    fig.add_trace(go.Scatter(x=x, y=entry.processed.average_entropy_signal+y_spacing*i, mode='markers', marker=dict(symbol='circle-open')))
    fig.add_trace(go.Scatter(x=x, y=entry.entropy_fit.eval(x=x)+y_spacing*i, mode='lines'))

fig

## Save to .itx file for final Figure in Igor

In [26]:
save_to_igor_itx(f'exports/uncorrected_transition.itx',
                 xs=[x]*6,
                 datas=[v0_0, v0_1, cold_fit.eval(x=x), vp, vm, hot_fit.eval(x=x)],
                 names=['uncorrected_trans_00', 'uncorrected_trans_01', 'uncorrected_trans_fit_cold', 'uncorrected_trans_vp', 'uncorrected_trans_vm', 'uncorrected_trans_fit_hot'])

save_to_igor_itx(f'exports/bumpy_entropy_data.itx',
                 xs=[x]*6,
                 datas = [entry.processed.average_entropy_signal for entry in entries[1:]] + [entry.processed.average_entropy_signal for entry in entries[1:]],
                 names=[f'bumpy_entropy_{i}' for i in range(len(entries[1:]))] + [f'bumpy_entropy_fit_{i}' for i in range(len(entries[1:]))]
)